In [ ]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Système de recommendations de films par regroupement<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction

<font class="answer">
    
J'ai choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle se compose de 100836 notes représentant 9742 films. Elle contient également les métadonnées sur 19 genres et des liens sur [TMDB](https://www.themoviedb.org) qui permettront d'augmemter son contenu.
    
Plus spécifiquement, (Voir https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html et décrire en détails le format)
    
    
    
L'objectif de ce projet est de contruire un système de recommentation de films en utilisant les techniques de regroupement vu dans le cours. Les détails du sytème seront abordés plus loins dans ce notebooks

In [1]:
#
# imports utilitaires
#

%matplotlib inline

import re
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA, TruncatedSVD

from tqdm.notebook import tqdm

In [2]:
#
# imports faisant partie de nos propres modules
#

import helpers as hlp
import helpers.dataset.MovieLens as mvl

from helpers.jupyter import display_html

# Prétraitement

<font class="answer">

TODO

In [3]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

In [4]:
#
# explorer movies
#
print("Movies")

display(mvl_dataset.movies)

types_ = mvl_dataset.movies.dtypes.to_frame()
types_.columns = ["Type"]
display(types_.T)

Movies


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


,movieId,title,genres
Type,int64,object,object


<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir des caractéritiques "atomiques".

In [13]:
def show_na_simple(dataframe):
    na_ = dataframe.isna().sum().to_frame()
    na_.columns = ["NA"]
    display(na_.T)

def split_title(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else np.NaN
    else:
        year = np.NaN
    
    return pd.Series({"title": title, "year": year})

title_year_re = re.compile(self.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title, args=(title_year_re,))

In [14]:
show_na_simple(title_year)

,title,year
NA,0,12


In [ ]:
# validation des modalites
print("Unique movieId:", mvl_dataset.movies.movieId.nunique())
print("Unique title:", mvl_dataset.movies.title.nunique())

In [ ]:
# valeurs nulles
show_na_simple(mvl_dataset.movies)

<font class="answer">
    
2 constats:
* *movieId* est effectivement un identifiant tel que décrit par MovieLens. Il sera mis utilisé comme index.
* *title* semble avoir des doublons.
    
*genre* sera manipuler un peu plus loin dans le notebook

# Exploration